# **In this notebook, we want to practice the LSTM with DQN. We applied the many-to-one model. The last action of the action sequence will be taken.**

*    **history_length = 15，每經過15天的交易，當做玩完一次遊戲** 
*    **features_size = 7, containing the difference in open, close, volume, high, low price of current day. 剩下2個參數紀錄 amount, position**
*    **The initial weight of LSTM is set to be identity and the bias is 0, expect the bias of the forget gate is 1. We force the forget gate to open for tranfering the previous information.**
*    **The initial hidden states are zeros.**

**Mainly modeified from**  
*    https://github.com/shivamakhauri04/TradingBot/blob/master/1_dqn.ipynb
*    https://github.com/JayChanHoi/value-based-deep-reinforcement-learning-trading-model-in-pytorch?fbclid=IwAR0QBYORviYp4z3jQI2x4_PM32ht8njfU4s3XtlJltRusycb5Q_H-6EOfA8  

**For the Prioritized experience replay part is modified from**
*    https://mofanpy.com/tutorials/machine-learning/reinforcement-learning/prioritized-replay/ (有中文說明)
*    https://github.com/rlcode/per

**For LSTM (Long Short-Term Memory)**
*    https://blog.csdn.net/Cyril_KI/article/details/122557880
*    https://wandb.ai/sauravmaheshkar/LSTM-PyTorch/reports/Using-LSTM-in-PyTorch-A-Tutorial-With-Examples--VmlldzoxMDA2NTA5
*    https://github.com/conditionWang/DRQN_Stock_Trading

**For GRU (Gated Recurrent Unit)**
*    https://ithelp.ithome.com.tw/articles/10194201

# **Connect to myDrive**

In [ ]:
from google.colab import drive
import os
import zipfile
drive.mount('/content/drive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Trading/Single Stock Trading/"
%cd "/content/drive/MyDrive/Trading/Single Stock Trading/"

Mounted at /content/drive
/content/drive/MyDrive/Trading/DQN for Single Stock Trading


# **Packages install**

In [ ]:
!pip install yfinance
import yfinance as yf

import json
from copy import deepcopy
import yaml
import random
import datetime
import time
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# **Data Preparation**

In [ ]:
'''
語法是 yf.download(
   股票代號,
   period=日期範圍 (1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max)
   interval=頻率 (1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo)
   )
或
語法是 yf.download(
   股票代號,
   start=開始日期,
   end=完結日期
   )
'''
stock_abbr = 'NVDA'
start = "2016-01-01"
end = "2022-05-20"

data = yf.download(stock_abbr, start=start, end=end)

'''
stock_abbr = 'NVDA'
start = datetime.datetime(2016,1,1)
end = datetime.datetime(2022,5,20)
time_interval = '1d'
data = pd.DataFrame()
sd = start
while sd < end:
    ed = sd + datetime.timedelta(days=7)
    tmp = yf.download(tickers=stock_abbr, start=sd, end=ed, interval=time_interval)
    data = pd.concat([data, tmp], axis=0)
    sd = ed
'''
data = data.drop(columns=["Adj Close"])

if data.isnull().sum().sum() != 0: 
    # 如果有缺資料要補齊，這裡目前沒有缺
    pass
data.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2016-01-04,8.0725,8.1450,8.0100,8.0925,35807600
2016-01-05,8.2450,8.3600,8.1250,8.2225,49027200
2016-01-06,8.0875,8.1250,7.7900,7.8825,44934400
2016-01-07,7.6850,7.7375,7.4700,7.5700,64530400
2016-01-08,7.6675,7.6750,7.3925,7.4075,39847200
2016-01-11,7.4150,7.4725,7.2875,7.4200,40937200
2016-01-12,7.5525,7.6550,7.4925,7.5450,46935600
2016-01-13,7.6025,7.6525,7.3075,7.3150,48167200
2016-01-14,7.1650,7.2500,6.9550,7.1675,60023600


In [ ]:
# the len of the testing set must coincide with the environment state
date_split = '2021-09-02'
train_data = data[:date_split]
test_data = data[date_split:]
len(train_data),len(test_data)

(1428, 180)

# **Environment**
environment只能提供給actor做決定的資訊  

In [ ]:
class Environment():
    def __init__(self, data, history_length):
        self.data = data
        self.history_length = history_length
        self.reset()

    def reset(self):
        self.t = 1
        self.profits = 0
        self.positions = []
        
        self.count_buy = 0
        self.count_sell_empty_position = 0
        self.count_p_reward = 0
        self.count_n_reward = 0
        self.count_hold = 0
        
        
        self.feature = []
        while self.t <= self.history_length:
            self.feature.append([0, 0] + list(self.data.iloc[self.t,:]))    # 第1天跟第0天，各個feature的變化量
            self.t+=1
        # return the state/observation representation
        return self.feature        # 前兩個存擁有的 股數 以及 市值
                          # 前面三個attribute存前一個action, buy: [0,1,0]; sell: [0,0,1], hold: [1,0,0]. (https://github.com/conditionWang/DRQN_Stock_Trading) 
                          # 最早的history_length都是hold，從history_length+1天開始才會有新的action
                          # the env.reset().size() --> [history_length, 8]
    
    def __call__(self, action):
        if action == 0:  # buy or hold
            if len(self.positions) == 0:    # 沒有股票
                self.positions.append(self.data.iloc[self.t, :]['Open'])
                self.count_buy+=1
                reward = 0
            else: 
                reward = 0
        
        elif action == 1:  # sell or hold
            if len(self.positions) != 0:    # 有股票可以賣
                profits = 0
                for p in self.positions:
                    profits += (self.data.iloc[self.t, :]['Open'] - p)
                # actually, this provides a way to make the reward be stochastic

                # in this case reward is deterministic
                if profits < 0:
                    reward = -1
                    self.count_n_reward+=1
                
                else:
                    reward = 1
                    self.count_p_reward+=1
                self.profits += profits
                self.positions = []
            else:
                reward = 0
        else: 
            raise ValueError('please input correct action 0:hold, 1:buy, 2:sell"')
        hold_size = len(self.positions)
        hold_amount = sum(self.positions)
        
        
        self.t += 1
        
        # 一個state裡，共 2 + 5 個 features
        self.feature.pop(0)
        self.feature.append([hold_size, hold_amount] + list(self.data.iloc[self.t,:])) # 各個feature的變化量
        
        return self.feature, reward, self.profits, self.count_p_reward, self.count_n_reward, hold_size, hold_amount

Example of how the environment works

In [ ]:
env = Environment(train_data,5)
state = env.reset()
print(state)
print("--------------------------------------------------------------------------------分隔線---------------------------------------------------------------------------------------------")
for act in [0,1,0,1]:
    print(env(act))

[[0, 0, 8.244999885559082, 8.359999656677246, 8.125, 8.22249984741211, 49027200.0], [0, 0, 8.087499618530273, 8.125, 7.789999961853027, 7.882500171661377, 44934400.0], [0, 0, 7.684999942779541, 7.737500190734863, 7.46999979019165, 7.570000171661377, 64530400.0], [0, 0, 7.667500019073486, 7.675000190734863, 7.392499923706055, 7.40749979019165, 39847200.0], [0, 0, 7.414999961853027, 7.472499847412109, 7.287499904632568, 7.420000076293945, 40937200.0]]
--------------------------------------------------------------------------------分隔線---------------------------------------------------------------------------------------------
([[0, 0, 8.087499618530273, 8.125, 7.789999961853027, 7.882500171661377, 44934400.0], [0, 0, 7.684999942779541, 7.737500190734863, 7.46999979019165, 7.570000171661377, 64530400.0], [0, 0, 7.667500019073486, 7.675000190734863, 7.392499923706055, 7.40749979019165, 39847200.0], [0, 0, 7.414999961853027, 7.472499847412109, 7.287499904632568, 7.420000076293945, 40937200.0

# **Model structure**

# **DQRN -- LSTM with DQN**

## **LSTM Input說明(GRU的狀況相同，但GRU盡量用rmsprop當optimizer，不要用adam, sgd)**

(https://blog.csdn.net/Cyril_KI/article/details/122557880)

**nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)**
*    input_size：在文本处理中，由于一个单词没法参与运算，因此我们得通过Word2Vec来对单词进行嵌入表示，将每一个单词表示成一个向量，此时input_size=embedding_size。比如每个句子中有五个单词，每个单词用一个100维向量来表示，那么这里input_size=100；在时间序列预测中，比如需要预测负荷，每一个负荷都是一个单独的值，都可以直接参与运算，因此并不需要将每一个负荷表示成一个向量，此时input_size=1。 但如果我们使用多变量进行预测，比如我们利用前24小时每一时刻的[负荷、风速、温度、压强、湿度、天气、节假日信息]来预测下一时刻的负荷，那么此时input_size=7。
*    hidden_size：隐藏层节点个数。可以随意设置。(就是neuro個數)
*    num_layers：層數
*    batch_first：默认为False，意义见后文

**self.lstm(input, (h_0, c_0))**
*    input.size() == (seq_len, batch_size, input_size) when batch_first=False, input.size() == (batch_size, seq_len, input_size) when batch_first=True.
*    seq_len：在文本处理中，如果一个句子有7个单词，则seq_len=7；在时间序列预测中，假设我们用前24个小时的负荷来预测下一时刻负荷，则seq_len=24。
*    h_0 = (num_directions * num_layers, batch_size, hidden_size_out) the initial hidden state for each element in the batch.
*    c_0 = (num_directions * num_layers, batch_size, hidden_size_cell) the initial cell state for each element in the batch.
*    num_directions：如果是双向LSTM，则num_directions=2；否则num_directions=1。

**self.lstm輸出 --> output, (h_0, c_0)**
*    output.size() == (seq_len, batch_size, num_directions * hidden_size) if batch_first=False, output.size() == (batch_size, seq_len, num_directions * hidden_size) otherwise.


In [ ]:
class DRQN(torch.nn.Module):
    def __init__(self, input_size):
        super(DRQN, self).__init__()
        self.input_size = input_size
        self.backbone = nn.Sequential(
          nn.Linear(input_size, 32),
          nn.ELU(),
          nn.Linear(32, 32),
          nn.ELU()
        )
        
        self.lstm = nn.LSTM(32, 32, 1, batch_first=True)
        
        self.linear = nn.Sequential(
          nn.Linear(32, 2),
          nn.ELU(),
        )
        self.apply(self._init_weights)

    
    def _init_weights(self, module): 
    # references for setting the intial weight
    #   https://gist.github.com/SauravMaheshkar/5704edf87c33ab09033dc9c0a10adaa1
    #   https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        if isinstance(module, nn.LSTM):
            # initial the weight of the lstm to identity matrix for 1 input and 3 gates

            weight = torch.Tensor([])
            for _ in range(4):
                weight = torch.cat([weight, torch.eye(32)], dim=0) 
            module.weight_ih_l0.data.copy_(weight)
            module.weight_hh_l0.data.copy_(weight)

            # set the bias of the forget get to one, zero otherwise
            bias = torch.cat([torch.zeros(32), torch.ones(32),torch.zeros(32),torch.zeros(32)], dim=0) 
            module.bias_ih_l0.data.copy_(bias)
            module.bias_hh_l0.data.copy_(bias)
        
# Data Flow Protocol:
# 1. network input shape: (batch_size, seq_length, num_features)
# 2. LSTM output shape: (batch_size, seq_length, hidden_size)
# 3. Linear input shape:  (batch_size * seq_length, hidden_size)
# 4. Linear output: (batch_size * seq_length, out_size)

    def forward(self, x, h, c):
        x = self.backbone(x)
        
        rnn_out, (new_h, new_c) = self.lstm(x, (h, c))

        batch_size, seq_len, mid_dim = rnn_out.shape
        x = rnn_out.contiguous().view(seq_len * batch_size, mid_dim)  # 相當於是有seq_len * batch_size個128維的vector
        x = self.linear(x)
        # 這個x.size() --> torch.tensor([seq_len * batch_size, 3]), 所以在選action的時候，要隔seq_len選
        return x, seq_len, new_h, new_c

  # initial the hidden state and the cell state for "lstm"
    def init_hidden_state(self, batch_size):
        return torch.zeros([1, batch_size, 32]), torch.zeros([1, batch_size, 32])
      

In [ ]:
# 確認lstm的initialization是work的
model = DRQN(7)
t=0
for param in model.parameters():
  if t in [4,5,6,7]:
    print(param.data.size())
    print(param.data)
  t+=1

torch.Size([128, 32])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([128, 32])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([128])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.

# **LSTM with Dueling DQN**

In [ ]:
class Dueling_DRQN(nn.Module):
    def __init__(self, input_size):
        super(Dueling_DRQN, self).__init__()
        self.input_size = input_size
        self.backbone = nn.Sequential(
          nn.Linear(input_size, 32),
          nn.ELU(),
          nn.Linear(32, 32),
          nn.ELU()
        )
        
        self.lstm = nn.LSTM(32, 32, 1, batch_first=True)
            
        self.state_value = torch.nn.Sequential(
            nn.Linear(32, 1),
            nn.ELU(),
        )
        self.advantage_value = torch.nn.Sequential(
            nn.Linear(32, 3),
            nn.ELU(),
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            # initial the weight of the lstm to identity matrix for 1 input and 3 gates

            weight = torch.Tensor([])
            for _ in range(4):
                weight = torch.cat([weight, torch.eye(32)], dim=0) 
            module.weight_ih_l0.data.copy_(weight)
            module.weight_hh_l0.data.copy_(weight)

            # set the bias of the forget get to one, zero otherwise
            bias = torch.cat([torch.zeros(32), torch.ones(32),torch.zeros(32),torch.zeros(32)], dim=0) 
            module.bias_ih_l0.data.copy_(bias)
            module.bias_hh_l0.data.copy_(bias)

# Data Flow Protocol:
# 1. network input shape: (batch_size, seq_length, num_features)
# 2. LSTM output shape: (batch_size, seq_length, hidden_size)
# 3. Linear input shape:  (batch_size * seq_length, hidden_size)
# 4. Linear output: (batch_size * seq_length, out_size)

    def forward(self, x, h, c):
        x = self.backbone(x)
        rnn_out, (new_h, new_c) = self.lstm(x, (h, c))
        
        batch_size, seq_len, mid_dim = rnn_out.shape
        x = rnn_out.contiguous().view(seq_len * batch_size, mid_dim)  # 相當於是有seq_len * batch_size個128維的vector
        

        state_value = self.state_value(x)
        state_value = state_value[seq_len-1:len(x):seq_len] 

        advantage_value = self.advantage_value(x)
        advantage_value = advantage_value[seq_len-1:len(x):seq_len] 

        advantage_mean = torch.Tensor.mean(advantage_value, dim=1, keepdim=True)   
        q_value = state_value.expand([-1, 3]) + (advantage_value - advantage_mean.expand([-1, 3]))   # expand說明:https://blog.csdn.net/guofei_fly/article/details/104467138
        return q_value, new_h, new_c  
    
    # initial the hidden state and the cell state for "lstm" or "gru"
    def init_hidden_state(self, batch_size):
        return torch.zeros([1, batch_size, 32]), torch.zeros([1, batch_size, 32])                                         

# **Actor (Policy)**

In [ ]:
def policy(model, state, h, c, epsilon, device):
    # ----------------------------- epsilon greedy policy ----------------------------------------------
    # random policy
    action  = np.random.randint(2)

    # greedy policy
    if np.random.rand() > epsilon:
        action, seq_len, new_h, new_c = model(torch.Tensor(np.array(state, dtype=np.float32)).unsqueeze(0).to(device), h.to(device), c.to(device))
        action = torch.argmax(action, dim=1, keepdim=True)
        action = action[0][0]
    return action, new_h, new_c
def shuffle_tensor(size, device):
    shuffle_index = torch.randperm(size).to(device)

    return shuffle_index

def resume(model, cuda, resume_checkpoint):
    print('=> loading checkpoint : "{}"'.format(resume_checkpoint))
    model_dict = model.state_dict()
    if not cuda:
        checkpoint = torch.load(resume_checkpoint, map_location=lambda storage, loc: storage)['state_dict']
        checkpoint = {k: v for k,v in checkpoint.items() if k in model_dict}
    else:
        checkpoint = torch.load(resume_checkpoint)['state_dict']
        checkpoint = {k: v for k,v in checkpoint.items() if k in model_dict}
    model_dict.update(checkpoint)
    model.load_state_dict(model_dict)

    return model

#  **Prioritized experience replay**

利TD_error = |q_target - q_eval|比較大表示train的不好，需要多train幾次，所以應該要增加從buffer裡面被取到的次數。

In [ ]:
class Memory:  # stored as ( s, a, r, s_ ) in SumTree
    e = 0.01
    a = 0.6
    beta = 0.4
    beta_increment_per_sampling = 0.001

    def __init__(self, capacity):
        self.tree = SumTree(capacity)
        self.capacity = capacity

    def _get_priority(self, error):
        return (np.abs(error) + self.e) ** self.a

    def add(self, error, sample):
        p = self._get_priority(error)
        self.tree.add(p, sample)          # p: td_error, sample-->tuple, the experience 

    def sample(self, n):
        batch_state = []
        batch_action = []
        batch_reward = []
        batch_observation = []
        batch_h = []
        batch_c = []
        batch_new_h = []
        batch_new_c = []
        

        idxs = []
        segment = self.tree.total() / n
        priorities = []

        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])

        for i in range(n):
            a = segment * i
            b = segment * (i + 1)

            s = random.uniform(a, b)
            (idx, p, data) = self.tree.get(s)
            priorities.append(p)
            batch_state.append(data[0])
            batch_action.append(data[1])
            batch_reward.append(data[2])
            batch_observation.append(data[3])
            batch_h.append(data[4])
            batch_c.append(data[5])
            batch_new_h.append(data[6])
            batch_new_c.append(data[7])
            
            idxs.append(idx)
            
        sampling_probabilities = priorities / self.tree.total()
        is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
        is_weight /= is_weight.max()

        batch_state = torch.Tensor(np.array(batch_state, dtype=np.float32))
        batch_action = torch.Tensor(np.array(batch_action, dtype=np.int32))
        batch_reward = torch.Tensor(np.array(batch_reward, dtype=np.int32))
        batch_observation = torch.Tensor(np.array(batch_observation, dtype=np.float32))
        batch_h = torch.Tensor(np.array(batch_h, dtype=np.float32))
        batch_c = torch.Tensor(np.array(batch_c, dtype=np.float32))
        batch_new_h = torch.Tensor(np.array(batch_new_h, dtype=np.float32))
        batch_new_c = torch.Tensor(np.array(batch_new_c, dtype=np.float32))

        return batch_state, batch_action, batch_reward, batch_observation, batch_h, batch_c, idxs, is_weight

    def update(self, idx, error):
        p = self._get_priority(error)
        self.tree.update(idx, p)

Buffer內的每個experience都有權重了，但如果每次都要由小到大排序，選最大權重的那個太花時間，所以用SumTree加速

In [ ]:
# SumTree
# a binary tree data structure where the parent’s value is the sum of its children
from copy import deepcopy
class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        self.n_entries = 0

    # update to the root node
    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    # find sample on leaf node
    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    # store priority and sample
    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = deepcopy(data)
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

        if self.n_entries < self.capacity:
            self.n_entries += 1

    # update priority
    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    # get priority and sample
    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])

# **Training**

In [ ]:
env = Environment(train_data,3)

h, c = model.init_hidden_state(1)
model = DRQN(7)
states = []   
actions = []
rewards = []
observations = []
h_s = []
c_s = []
new_h_s = []
new_c_s = []
env.global_reset()
h, c = model.init_hidden_state(1)
state, done = env.reset()
while done==False:
    action, new_h, new_c = policy(model, state, h, c, 0.001, "cpu")
    observation, reward, done = env(action)
    
    # add the one new experience to memory buffer
    states.append(state[0])
    actions.append(action.item())
    rewards.append(reward)
    observations.append(observation[0])
    h_s.append(h.data[0][0].tolist())
    c_s.append(c.data[0][0].tolist())
    new_h_s.append(new_h.data[0][0].tolist())
    new_c_s.append(new_c.data[0][0].tolist())
    state = deepcopy(observation)
# state:[seq_len, features]; actions:[seq_len]; rewards:[seq_en]; observations:[seq_len, features]; h_s, c_s, new_h_s, new_c_s:[seq_len, 32]
experience = (states, actions, rewards, observations, h_s, c_s, new_h_s, new_c_s)


In [ ]:
def list2tensor(state):
    return torch.Tensor(np.array(state, dtype=np.float32)).unsqueeze(0)

def rl_agent_train(model, env, memory, PER, num_epoch, step_max, epsilon, device, memory_size, train_freq, batch_size, mode, model_ast,
                   discount_rate, optimizer, update_model_ast_freq, epsilon_min, epsilon_reduce_freq, epsilon_reduce,
                   rewards, losses, save_freq, checkpoint_dir, global_step):
    prg_bar = tqdm(range(num_epoch))
    model.train()  # 訓練前，先確保 network 處在 training 模式

    epsilon = epsilon - int(global_step/epsilon_reduce_freq)*epsilon_reduce   # 要特別注意當訓練模型中斷要回復時，要記得把epsilon也更新，不然greedy-policy會變成從頭開始亂選
    if epsilon < epsilon_min:
      epsilon = epsilon_min

    for epoch in prg_bar:
        env.global_reset()
        h, c = model.init_hidden_state(1)
        step = 0
        total_loss = 0
        total_reward = 0

        # add replay memory buffer for the agent
        while step < step_max:
            state, done = env.reset()
            state = deepcopy(state)
            
            states = []   
            actions = []
            rewards = []
            observations = []
            h_s = []
            c_s = []
            new_h_s = []
            new_c_s = []
            while done==False:
                action, new_h, new_c = policy(model, state, h, c, epsilon, device)
                observation, reward, done = env(action)

                
                # add the one new experience to memory buffer
                states.append(state[0])
                actions.append(action.item())
                rewards.append(reward)
                observations.append(observation[0])
                h_s.append(h.data[0][0].tolist())
                c_s.append(c.data[0][0].tolist())
                new_h_s.append(new_h.data[0][0].tolist())
                new_c_s.append(new_c.data[0][0].tolist())
                state = deepcopy(observation)
            # list --> 
            #   state:[seq_len, features]; 
            #   actions:[seq_len]; 
            #   rewards:[seq_en]; 
            #   observations:[seq_len, features]; 
            #   h_s, c_s, new_h_s, new_c_s:[seq_len, 32]
            experience = (states, actions, rewards, observations, h_s, c_s, new_h_s, new_c_s)


            # 做完一次length的交易，calculate the td_error of this length for Prioritized Experience Replay
            # hence, the state_size=[length, feature_size]
            if PER == True:               
                
                q_eval, seq_len, _, _ = model(list2tensor(states).to(device), list2tensor(h_s).to(device), list2tensor(c_s).to(device))
                q_eval = q_eval[seq_len-1:q_eval.size()[0]:seq_len]
                q_eval = q_eval.gather(1,list2tensor(actions).long().to(device)).squeeze(0)
                

                q_next, seq_len, _, _ = model_ast(list2tensor(observations).to(device), list2tensor(new_h_s).to(device), list2tensor(new_c_s).to(device)) 
                q_next = q_next[seq_len-1:q_next.size()[0]:seq_len]
                

                if mode == 'dqn':
                    q_target = reward + discount_rate * q_next.max(1)[0]
                
                elif mode == "doubledqn":
                    q_target, seq_len, _, _ = model(list2tensor(observations).to(device), list2tensor(new_h_s).to(device), list2tensor(new_c_s).to(device))
                    q_target = q_target[seq_len-1:q_target.size()[0]:seq_len]
                    q_target = reward + discount_rate * q_next.gather(1, torch.argmax(q_target, dim=1, keepdim=True))
                    q_target = q_target.squeeze(0)
                
                elif mode == 'duelingdqn':
                    q_target, seq_len, _, _ = model(list2tensor(observations).to(device), list2tensor(new_h_s).to(device), list2tensor(new_c_s).to(device))
                    q_target = q_target[seq_len-1:q_target.size()[0]:seq_len]
                    q_target = reward + discount_rate * q_next.gather(1, torch.argmax(q_target, dim=1, keepdim=True))
                    q_target = q_target.squeeze(0)
                
                else:
                    raise ValueError('please input correct mode for rl agent, either "dqn", "doubledqn", or "duelingdqn"')
                
                td_error = torch.abs(q_target-q_eval).data.cpu().numpy()
                memory.add(td_error, experience)
            
            else:
                memory.add(np.array([1],dtype="float32"), experience)

            if memory.tree.n_entries == memory_size:
            #-------------------------------training part---------------------------------
                if global_step % train_freq == 0:       # 當buffer滿了之後，不希望每做一次action就更新一次參數，所以用一個train_freq=10控制
                    # 要從memory buffer取一個barch_size的experience出來
          
                    batch_state, batch_action, batch_reward, batch_observation, batch_h, batch_c, batch_new_h, batch_new_c, idxs, is_weight = memory.sample(batch_size)   
                    
                    batch_state = batch_state.to(device)
                    batch_action = batch_action.to(device)
                    batch_reward = batch_reward.to(device)
                    batch_observation = batch_observation.to(device)
                    batch_h = batch_h.to(device)    # length = batch_size
                    batch_c = batch_c.to(device)  #   length = batch_size
                    batch_new_h = batch_h.to(device)  # length = batch_size
                    batch_new_c = batch_c.to(device)  # length = batch_size

                    
                    q_evals, seq_len, _, _ = model(batch_state, batch_h.to(device), batch_c.to(device))
                    q_evals = q_evals[seq_len-1:q_evals.size()[0]:seq_len]
                    q_evals = q_evals.gather(1, batch_action.long().unsqueeze(1))            # model(batch_state).size()-->torch.Size([batch_size, 3])
                                                                # batch_action.long().unsqueeze(1).size()-->torch.Size([batch_size, 1])
                                                                # 利用gather選出model(batch_state)所對應的值，所以q_eval.size()-->torch.Size([batch_size, 1]) 
                                                                # gather說明:https://zhuanlan.zhihu.com/p/352877584
                    
                    # update with all sequence
                    q_nexts, seq_len, _, _ = model_ast(batch_observation, batch_new_h.to(device), batch_new_c.to(device))       # 對batch內每一個state產出對應的action，所以q_next.size()-->torch.Size([batch_size, 3])
                    q_nexts = q_nexts[seq_len-1:q_nexts.size()[0]:seq_len]  
                    q_nexts = q_nexts.detach()

                    if mode == 'dqn':
                        q_targets = batch_reward + discount_rate * q_nexts.max(1)[0]
                        q_targets = q_targets.unsqueeze(1)
                    elif mode == "doubledqn":
                        # update with all sequence
                        q_targets, seq_len, _, _ = model(batch_observation, batch_new_h.to(device), batch_new_c.to(device))                        
                        q_targets = batch_reward.unsqueeze(1) + discount_rate * q_nexts.gather(1, torch.argmax(q_targets, dim=1, keepdim=True))
                    elif mode == 'duelingdqn':
                        h, c = model.init_hidden_state(batch_size)
                        # update with all sequence
                        q_targets, seq_len, _, _ = model(batch_observation, batch_new_h.to(device), batch_new_c.to(device))
                        q_targets = batch_reward.unsqueeze(1) + discount_rate * q_nexts.gather(1, torch.argmax(q_targets, dim=1, keepdim=True))
                    else:
                        raise ValueError('please input correct mode for rl agent, either "dqn", or "ddqn"')
                    
                    
                    optimizer.zero_grad()
                    # 對取出來的 batch 計算新的td_errors以及update priority，還有loss要做weighted mse
                    if PER == True:
                        td_errors = torch.abs(q_targets-q_evals).data.cpu().numpy()
                        for i in range(batch_size):
                            idx = idxs[i]        
                            memory.update(idx, td_errors[i])
                        loss = torch.dot(torch.FloatTensor(is_weight).to(device),((q_evals-q_targets)**2).squeeze(1))/torch.FloatTensor(is_weight).to(device).sum()
                    
                    else:
                        loss = F.mse_loss(q_evals, q_targets)
                    
                    
                    total_loss += loss.item()
                    loss.backward()
                    optimizer.step()
                #-------------------------------end training part---------------------------------

                #-------------------------------update Q_ast part----------------------------------
                if global_step % update_model_ast_freq == 0:       # update_model_ast_freq=500 才更新一次 Q_ast function  
                    para = {k: 0.1 * v + 0.9 * model.state_dict()[k] for k, v in model_ast.state_dict().items()}   # 更新 model_ast，保留0.3的model_ast存下0.7迭代的model
                    model_ast.load_state_dict(para)
                    
                    
                # epsilon
                if epsilon > epsilon_min and global_step % epsilon_reduce_freq == 0:
                    epsilon -= epsilon_reduce

            total_reward += reward
            state = deepcopy(observation)
            h = deepcopy(new_h)
            c = deepcopy(new_c)
            step += 1
            global_step += 1     # global_step是額外用來控制要訓練頻率的，跟step不同
        rewards.append(total_reward)
        losses.append(total_loss)


        print('------------------------------------------epoch {}--------------------------------------------'.format(epoch))
        
        print("buy_days : {}".format(buy))
        print("poitive_reward : {}".format(positive))
        print("negative_reward : {}".format(negative))
        print("hold_days : {}".format(hold))

        print('total loss : {}'.format(total_loss))                        
        print('total reward : {}'.format(total_reward))                # 經過一個episode的total_reward，
        # print('-----------------------------------------------------------------------------------------------')
        prg_bar.set_description(f"Total Reward: {total_reward: 4.1f}, Total Loss: {total_loss: 4.1f}")

        if (epoch + 1) % save_freq == 0:
            checkpoint_state = {'epoch': epoch, 'state_dict': model.state_dict()}
            if not os.path.isdir(checkpoint_dir):
                os.mkdir(checkpoint_dir)
            torch.save(checkpoint_state, os.path.join(checkpoint_dir, '{}_checkpoint.pth.tar'.format(mode+"_"+str(epoch)+'_'+str(global_step))))
    
            # save losses and rewards
            with open(os.path.join(checkpoint_dir, "{}_losses.json".format(mode+"_"+str(epoch)+'_'+str(global_step))), "w") as fp:
                json.dump(losses, fp, indent=2) 
            with open(os.path.join(checkpoint_dir, "{}_rewards.json".format(mode+"_"+str(epoch)+'_'+str(global_step))), "w") as fp:
                json.dump(rewards, fp, indent=2) 

    return rewards, losses

In [ ]:
def train(env, model, config):
    cuda = torch.cuda.is_available()
    if torch.cuda.is_available():
        device = "cuda"
        print('------------------------Train with cuda------------------------------------')
    else:
        device = "cpu"
        print('------------------------Train with cpu------------------------------------')
    model.to(device)
    model_ast = type(model)(model.input_size).to(device)
    
    
    if os.path.exists(os.path.join(config["checkpoint_dir"], config['resume_checkpoint'])):
        model = resume(model=model, cuda=cuda, resume_checkpoint=os.path.join(config["checkpoint_dir"], config['resume_checkpoint']))
        model_ast = type(model)(model.input_size).to(device)
        print('------------------------train from checkpoint------------------------------------')
    else:
        print('checkpoint: "{}" does not exist'.format(os.path.join(config["checkpoint_dir"], config['resume_checkpoint'])))
        print('------------------------train from beginning------------------------------------')
    
    
    if torch.cuda.device_count() > 1:
        print(" let's use , {} GPU".format(torch.cuda.device_count()))
        model = torch.nn.DataParallel(model)
        model_ast = torch.nn.DataParallel(model_ast)
    model.train()
    model_ast.train(mode=False)
    
    if config["optimizer"] == "Adam":
        optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'], weight_decay=0.0005)
    elif config["optimizer"] == "RMSprop":
        optimizer = torch.optim.RMSprop(params=model.parameters(), lr=config['learning_rate'], alpha=0.9)
    
    memory = Memory(config['memory_size'])
    
    rewards = []
    losses = []

    rewards, losses = rl_agent_train(
        model=model,
        model_ast=model_ast,
        env=env,
        memory=memory,
        PER=config["per"], 
        step_max=len(env.data)-config["history_length"]-2,
        epsilon=config['epsilon'],
        epsilon_min=config['epsilon_min'],
        epsilon_reduce=config['epsilon_reduce'],
        epsilon_reduce_freq=config['epsilon_reduce_freq'],
        device=device,
        memory_size=config['memory_size'],
        global_step=config["global_step"],
        train_freq=config['train_freq'],
        batch_size=config['batch_size'],
        discount_rate=config['discount_rate'],
        optimizer=optimizer,
        losses=losses,
        rewards=rewards,
        update_model_ast_freq=config['update_model_ast_freq'],
        checkpoint_dir=os.path.join(config['checkpoint_dir']),
        mode=config['mode'],
        save_freq=config['save_freq'],
        num_epoch=config['num_epoch']
    )

    return model, losses, rewards

# **Parameters Setting**
以下參數要記得改
*    config["checkpoint_dir"]
*    config["global_step"]
*    config["optimizer"]
*    config["learning_rate"]
*    config["mode"]
*    config["per"]
*    function main 裡的 model

In [ ]:
config = {
    "resume_checkpoint": "None",   # 放之前存的 "checkpoint" 檔名 or "None"
    "global_step": 0,
    "checkpoint_dir": 'Dueling_DRQN LSTM PER checkpoint',
    "history_length": 15,       # the number of seq_len
    #------------------------train spec-----------------------
    "learning_rate": 0.00025,             # small:0.00025 for adam, larger for non momentum sgd variant; larger for rmsprop: 0.0025
    "optimizer": "Adam",               # "Adam" or "RMSprop"
    "epsilon": 1.0,
    "epsilon_min": 0.1,
    "epsilon_reduce": 0.0005,
    "epsilon_reduce_freq": 600,    # 利用global_step控制
    "memory_size": 640,
    "train_freq": 15,         # 利用global_step控制
    "batch_size": 32,        
    "discount_rate": 0.79,
    "update_model_ast_freq":600,    # 利用global_step控制
    "mode": 'duelingdqn',          # 'dqn', "doubledqn", "duelingdqn"
    "per": True,           # applying the Prioritized Experience Replay or not
    "save_freq": 10,
    "num_epoch": 500
}

# **Main**

In [ ]:
def main(config, train_data, test_data):
    SEED = 100
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)
    model = Dueling_DRQN(10)         # DuelingDRQN or DRQN           
    train_env = Environment(train_data, config["history_length"])
    test_env = Environment(test_data, config["history_length"])   
    model, losses, rewards = train(env=train_env, model=model, config=config)
    return losses, rewards

if __name__ == "__main__":
    print(config)
    losses, rewards = main(config, train_data, test_data)
    
    
    

{'resume_checkpoint': 'None', 'global_step': 0, 'checkpoint_dir': 'Dueling_DRQN LSTM PER checkpoint', 'history_length': 15, 'learning_rate': 0.00025, 'optimizer': 'Adam', 'epsilon': 1.0, 'epsilon_min': 0.1, 'epsilon_reduce': 0.0005, 'epsilon_reduce_freq': 600, 'memory_size': 640, 'train_freq': 15, 'batch_size': 32, 'discount_rate': 0.79, 'update_model_ast_freq': 600, 'mode': 'duelingdqn', 'per': True, 'save_freq': 10, 'num_epoch': 500}
------------------------Train with cpu------------------------------------
checkpoint: "Dueling_DRQN LSTM PER checkpoint/None" does not exist
------------------------train from beginning------------------------------------


  0%|          | 0/500 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rewards, c='tab:red')
plt.title('Learning curve of rewrads')

In [ ]:
plt.plot(losses, c='tab:blue')
plt.title('Learning curve of losses')